# Built-In Context Manager

This notebook is very similar to [`single_hr_notebook_example.ipynb`](examples/single_hr_notebook_example.ipynb). 

The purpose of this notebook is to demonstrate using the BleakModel context manager to use `with` blocks to more robustly and pythonically disconnect/clean up blueooth accessories before the program exits.

In [ ]:
import asyncio
from bleak_fsm import BleakModel
from pycycling.heart_rate_service import HeartRateService

model = BleakModel(connection_timeout=20)
model.wrap = lambda client: HeartRateService(client)
model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
def handle_hr_measurement(value):
    print("Using Pycycling wrapper around BleakClient")
    print(f"Heart Rate: {value}")
model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)

In [ ]:
await BleakModel.start_scan()
await asyncio.sleep(3)
await BleakModel.stop_scan()

In [ ]:
BleakModel.bt_devices

In [ ]:
target_name = "WHOOPDEDOO" # Change this to the name of your device

In [ ]:
target_address = ""
for address, (ble_device, advertisement_data) in BleakModel.bt_devices.items():
    if ble_device.name == target_name:
        target_address = address
        print(f"Found {target_name} at {target_address}")

## The Main Part: `with` Block:

In [ ]:
async with model:
    await model.set_target(target_address)
    await model.connect()
    await model.stream()
    await asyncio.sleep(10)

In [ ]:
# verify that the model is in its initial state
assert model.state == "Init"

The above `with` block just ensures that when the block ends, `model.clean_up()` is called.

The [source of `model.clean_up`](https://github.com/tensorturtle/bleak-fsm/blob/a9a4ac165aa0adc1425c6d7a7fbabb04ca4be727/bleak_fsm/bleak_model.py#L150) shows that it calls `model.disconnect()` and/or `model.unset_target()` depending on the current state.

So in effect, the above code is equivalent to:

In [ ]:
try:
    await model.set_target(target_address)
    print(f"State after setting target: {model.state}")
    await model.connect()
    print(f"State after connecting: {model.state}")
    await model.stream()
    print(f"State after streaming: {model.state}")
    await asyncio.sleep(10)

    await model.disconnect()
    await model.unset_target()
    # or, `clean_up()` will smartly disconnect and unset target

finally:
    await model.clean_up()
    print("Disconnected and stopped")

In [ ]:
# again, check that the model is in its initial state
assert model.state == "Init"
